In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [3]:
from typing import Dict
from acctrack.io.athena_raw_root import AthenaRawRootReader
from acctrack.utils.utils_plot import add_mean_std, create_figure
from acctrack.io import utils_athena_raw_root as utils_raw_root
from acctrack.task.convert_gnn_tracks_for_fitting import ConvertGNNTracksForFitting

import awkward as ak
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
from pathlib import Path

In [47]:
base_dir = Path("/home/xju/projects/tracking/integrateToAthena/run_21.9.26/RunAlexisInputs/data")
file_reco = base_dir / "trackReco_singleCutFilter-wrangler_events_1681-1780_thr.h5"
origin_evtid_matching_fname = base_dir / "correspondence.txt"
origin_rdo_matching_fname = base_dir / "ttbar_job_inputfiles.txt"

evtid_matching_fname = base_dir / "evtid_matching.csv"
rdo_matching_fname = base_dir / "rdo_matching.csv"

# Reformat the inputs

In [48]:

evtid_map_to_event_number = pd.read_csv(origin_evtid_matching_fname, sep='\s+', header=None,
                       names=["evtID", "orgEvtID", "rdoEvtID"])
evtid_map_to_event_number["rdoNum"] = 0


In [49]:
evtid_map_to_event_number.iloc[1600:1800]

,evtID,orgEvtID,rdoEvtID,rdoNum
1600,1601,242020,77118,0
1601,1602,242020,77138,0
1602,1603,242020,77136,0
1603,1604,242020,77142,0
1604,1605,242020,77162,0
...,...,...,...,...
1795,1796,242020,89863,0
1796,1797,242020,89909,0
1797,1798,242020,89914,0
1798,1799,242020,89931,0


In [ ]:

start_event_id = 1681
end_event_id = 1780
evtid_map_to_event_number = evtid_map_to_event_number.iloc[start_event_id:end_event_id]
evtid_map_to_event_number.to_csv(evtid_matching_fname, sep='\t', header=False, index=False)


rdo_info = pd.read_csv(
    origin_rdo_matching_fname, sep='\s+', header=None,
    names=["rodNum", "rdoFileName"])
rdo_info[['rdoFileName', 'rodNum']].to_csv(rdo_matching_fname, sep='\t', header=None, index=False)

In [46]:
evtid_map_to_event_number

,evtID,orgEvtID,rdoEvtID,rdoNum
1681,1682,242020,24239,0
1682,1683,242020,24222,0
1683,1684,242020,24253,0
1684,1685,242020,24250,0
1685,1686,242020,24283,0
...,...,...,...,...
1776,1777,242020,87509,0
1777,1778,242020,87520,0
1778,1779,242020,87527,0
1779,1780,242020,87571,0


In [6]:
requested_event_numbers = np.sort(evtid_map_to_event_number.rdoEvtID.values)
requested_event_numbers

array([22222, 22241, 22249, 22256, 22266, 22272, 22295, 22320, 22339,
       22389, 24222, 24239, 24250, 24253, 24283, 24312, 24314, 24359,
       24419, 25223, 25232, 25235, 25274, 25277, 25290, 25293, 25321,
       25332, 25362, 26221, 26234, 26241, 26242, 26260, 26269, 26274,
       26278, 26296, 26303, 27141, 27153, 27166, 27181, 27197, 27200,
       27216, 27228, 27243, 27273, 28145, 28153, 28166, 28176, 28199,
       28200, 28205, 28207, 28228, 28234, 28652, 28657, 28666, 28668,
       28676, 28703, 28728, 28757, 28759, 28793, 29151, 29153, 29161,
       29173, 29180, 29191, 29211, 29231, 29262, 29299, 29651, 29656,
       29673, 29676, 29689, 29704, 29706, 29733, 29754, 29755, 87451,
       87457, 87462, 87467, 87484, 87490, 87509, 87520, 87527, 87571,
       88306])

In [26]:
convert = ConvertGNNTracksForFitting(evtid_matching_fname, rdo_matching_fname,
                                     file_reco, "", base_dir/"forXiangyang", max_evts=end_event_id+1)

In [44]:
convert.read_evt_info()

In [45]:
convert.evt_info.evtID.values.tolist()

[1682,
 1683,
 1684,
 1685,
 1686,
 1687,
 1688,
 1689,
 1690,
 1691,
 1692,
 1693,
 1694,
 1695,
 1696,
 1697,
 1698,
 1699,
 1700,
 1701,
 1702,
 1703,
 1704,
 1705,
 1706,
 1707,
 1708,
 1709,
 1710,
 1711,
 1712,
 1713,
 1714,
 1715,
 1716,
 1717,
 1718,
 1719,
 1720,
 1721,
 1722,
 1723,
 1724,
 1725,
 1726,
 1727,
 1728,
 1729,
 1730,
 1731,
 1732,
 1733,
 1734,
 1735,
 1736,
 1737,
 1738,
 1739,
 1740,
 1741,
 1742,
 1743,
 1744,
 1745,
 1746,
 1747,
 1748,
 1749,
 1750,
 1751,
 1752,
 1753,
 1754,
 1755,
 1756,
 1757,
 1758,
 1759,
 1760,
 1761,
 1762,
 1763,
 1764,
 1765,
 1766,
 1767,
 1768,
 1769,
 1770,
 1771,
 1772,
 1773,
 1774,
 1775,
 1776,
 1777,
 1778,
 1779,
 1780,
 1781]

In [25]:
convert.run()

1682 does not have processed data, no sortting
1683 does not have processed data, no sortting
1684 does not have processed data, no sortting
1685 does not have processed data, no sortting
1686 does not have processed data, no sortting
1687 does not have processed data, no sortting
1688 does not have processed data, no sortting
1689 does not have processed data, no sortting
1690 does not have processed data, no sortting
1691 does not have processed data, no sortting
1692 does not have processed data, no sortting
1693 does not have processed data, no sortting
1694 does not have processed data, no sortting
1695 does not have processed data, no sortting
1696 does not have processed data, no sortting
1697 does not have processed data, no sortting
1698 does not have processed data, no sortting
1699 does not have processed data, no sortting
1700 does not have processed data, no sortting
1701 does not have processed data, no sortting
1702 does not have processed data, no sortting
1703 does not

In [13]:
# recoTracks = [dict() for i in range(0, 1800)]
recoTracks = dict()

methods = ["singleCutFilter", "wrangler"]
print("Getting reco tracks from file") 
# taken from https://gitlab.cern.ch/gnn4itkteam/tracks_reconstruction/-/blob/master/truthMatch.py#L59
with pd.HDFStore(file_reco, mode='r') as reader:
    for eventId in range(0, 2000):
        for m in methods:
            dataname = "/event{0}/{1}/reco_tracks".format(eventId, m)
            df_trks = reader.get(dataname)
            # Remove -1 that are placeholders for empty hit
            trks = df_trks.values
            trks = [list(filter(lambda x: x !=-1, trk)) for trk in trks]
            if eventId in recoTracks:
                recoTracks[eventId].update( { m : trks })
            else:
                recoTracks[eventId] = { m : trks }

Getting reco tracks from file


In [39]:
evtid = 1682
all_track_candidates = recoTracks[evtid]["wrangler"] + recoTracks[evtid]["singleCutFilter"]

In [40]:
len(all_track_candidates)

7258

In [41]:
all_track_candidates[0]

[22783, 22407, 796, 22654, 22302, 572, 22520, 22931, 643, 721]

In [35]:
good_trks = [x for x in all_track_candidates if len(x) >= 3]

In [36]:
len(good_trks)

2594

In [37]:
len(good_trks[0]), len(good_trks[1])

(10, 15)

In [38]:
good_trks[0]

[22783, 22407, 796, 22654, 22302, 572, 22520, 22931, 643, 721]